# Components Notebook
The purpose of this notebook is to locally test the individual pipeline components before running the pipelines in google cloud

In [ ]:
import os
import google.cloud.aiplatform as vertex_ai

In [ ]:
GCP_PROJECT_ID = "jokes"
GCP_REGION = "northamerica-northeast1"
GCP_STORAGE_BUCKET = "gs://vertex_ai_experiments"

In [ ]:
vertex_ai.init(
    project=GCP_PROJECT_ID,
    location=GCP_REGION,
    staging_bucket=GCP_STORAGE_BUCKET
)

## Training Pipeline Components
1. Build Training Image
2. Data Preperation
3. Model Training

### Data Preperation

In [1]:
# Set required input parameters for running the component
DATASET_NAME = 'CIFAR10'
TRAIN_PATH = './data/train'
TEST_PATH = './data/test'
DOWNLOAD = True

In [2]:
! python3 ./pipeline/training/data_prep/task_data_prep.py \
--dataset_name $DATASET_NAME \
--train_path $TRAIN_PATH \
--test_path $TEST_PATH \
--download $DOWNLOAD

/opt/homebrew/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/Resources/Python.app/Contents/MacOS/Python: can't open file '/Users/tristano/Desktop/implementations/ViT/notebooks/./pipeline/training/data_prep/task_data_prep.py': [Errno 2] No such file or directory


### Model Training Component

In [ ]:
MODEL_NAME = "vit"
EXPERIMENT_NAME = "vit-experiment"
DEVICE = "cpu"
EPOCHS = 1
BATCH_SIZE = 32
WORKERS = 1
LEARNING_RATE = 0.01
DECAY = 0.08

In [ ]:
! python3 ./pipeline/training/model_training/task_train_model.py \
--model_name $MODEL_NAME \
--experiment_name $EXPERIMENT_NAME \
--device $DEVICE \
--epochs $EPOCHS 

# Inference Pipeline - Steps
1. Build serving image
2. Model Upload: registers model in Vertex AI
3. Create Endpoint: creates a Vertex AI endpoint
4. Model Deploy: deploys model to endpoint

## Build Serving Image

In [ ]:
SERVING_IMAGE_REPO = "vit-inference"
SERVING_IMAGE_TAG = "latest"
SERVING_IMAGE_URI = f"{GCP_REGION}-docker.pkg.dev/{GCP_PROJECT_ID}/{SERVING_IMAGE_REPO}/{SERVING_IMAGE_TAG}"

In [ ]:
! docker build -t $SERVING_IMAGE_URI -f ./dockerfiles/inference/Dockerfile

In [ ]:
! docker push $SERVING_IMAGE_URI

## Model Upload

In [ ]:
MODEL_ARTIFACT_URI = "gs://training-pipeline/model"
HEALTH_ROUTE = "/ping"
PREDICT_ROUTE = f"/predictions/{MODEL_NAME}"

In [ ]:
! python3 ./pipeline/inference/model_upload/run_model_upload_task.py \
--model_display_name $MODEL_NAME \
--model_artifact_uri $MODEL_ARTIFACT_URI \
--serving_image_uri $SERVING_IMAGE_URI \
--health_route $HEALTH_ROUTE \
--predict_route $PREDICT_ROUTE 

## Endpoint Create

In [ ]:
ENDPOINT_DISPLAY_NAME = "vit-endpoint"

In [ ]:
! python3 ./pipeline/inference/endpoint/endpoint_create.py \
--endpoint_display_name $ENDPOINT_DISPLAY_NAME

## Model Deployment

In [ ]:
model_artifact = vertex_ai.Model()
machine_type = "n1-standard-2"
accelerator_type = "ACCELERATOR_TYPE_UNSPECIFIED"
accelerator_count = "0"
traffic_percentage = "100"
autoscaling_nodes_min = "1"
autoscaling_nodes_max = "1"
autoscaling_cpu_utilization = "60"

In [ ]:
endpoint = vertex_ai.Endpoint()

In [ ]:
endpoint.deploy(
    model=model_artifact,
    deployed_model_display_name=MODEL_NAME,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    traffic_percentage=traffic_percentage,
    min_replica_count=autoscaling_nodes_min,
    max_replica_count=autoscaling_nodes_max,
    autoscaling_target_cpu_utilization=autoscaling_cpu_utilization
)